In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torchvision
import seaborn as sns
from PIL import Image
%matplotlib inline
sns.set_context('poster')
sns.set_color_codes()
import torchvision.transforms as T
from google.colab import drive
#drive.mount("/content/drive/")

#orig_path = '/content/drive/My Drive/bird_classification/' + '/data/bird_dataset/'

## For reproductibility
torch.manual_seed(2)
np.random.seed(2)
random.seed(2)

In [2]:
torchvision.__version__

'0.8.1+cu101'

In [ ]:
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
model.cuda()
model.eval()

In [ ]:
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

def get_prediction(img_path, threshold):
  img = Image.open(img_path)
  transform = T.Compose([T.ToTensor()])
  img = transform(img).cuda()
  pred = model([img])
  pred_score = list(pred[0]['scores'].cpu().detach().numpy())
  pred_t = [pred_score.index(x) for x in pred_score if x>threshold][-1]
  pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].cpu().detach().numpy())]
  pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].cpu().detach().numpy())]
  pred_boxes = pred_boxes[:pred_t+1]
  pred_class = pred_class[:pred_t+1]
  return pred_boxes, pred_class

def crop_image(img_path, threshold):
  boxes, classes = get_prediction(img_path, threshold)
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  cropped = []
  for i in range(len(classes)):
    if classes[i] is 'bird':
      coordinates = boxes[i]
      y = coordinates[0]
      x = coordinates[1]
      x1,y1 = int(y[0]), int(y[1])
      x2, y2 = int(x[0]), int(x[1])
      img_crop = img[y1:y2, x1:x2]
      cropped.append(img_crop)
  return cropped
  




In [ ]:
#path = orig_path + 'train_images/034.Gray_crowned_Rosy_Finch/Gray_Crowned_Rosy_Finch_0076_27200.jpg'
path = orig_path + 'test_images/mistery_category/fef53a1dada4a77de35c609180d41936.jpg'
lst = crop_image(path, 0.5)

## Crop test_file

In [ ]:
path = 'test_images/mistery_category/'
new_path = 'test_images_cropped/mystery_category/'
un_cropped = []
duplicates = []
for namefile in os.listdir(orig_path + path):
  current_img = orig_path + path + namefile
  try:
    cropped = crop_image(current_img, 0.55)
  except IndexError:
    print(f'No Bird detected on image {namefile}, trying lower threshold')
    try:
      cropped = crop_image(current_img, 0.40)
    except IndexError:
      un_cropped.append(current_img)
      continue
  if len(cropped) < 1:
    try:
      cropped = crop_image(current_img, 0.40)
    except IndexError:
      un_cropped.append(current_img)
      continue
    if len(cropped) < 1:
      un_cropped.append(current_img)
      continue 
  if len(cropped) > 1:
    print(f'Two bird detected on image {namefile}, trying bigger threshold')
    try:
      cropped = crop_image(current_img, 0.70)
    except IndexError:
      try:
        cropped = crop_image(current_img, 0.63)
      except IndexError:
        duplicates.append(current_img)
        continue
    if len(cropped) > 1:
      duplicates.append(current_img)
      continue
    elif len(cropped) < 1:
      cropped = crop_image(current_img, 0.63)
      if len(cropped) < 1 or len(cropped) > 2:
        duplicates.append(current_img)
        continue
  IMG = cropped[0]
  IMG = cv2.resize(IMG, (224,224), interpolation=cv2.INTER_LANCZOS4)
  cv2.imwrite(orig_path + new_path + namefile, IMG)

In [ ]:
duplicates

['/content/drive/My Drive/bird_classification//data/bird_dataset/test_images/mistery_category/03b5b072b80ec04df1dee668f44abad4.jpg',
 '/content/drive/My Drive/bird_classification//data/bird_dataset/test_images/mistery_category/04c2ed055c16441bc123f341ff172128.jpg',
 '/content/drive/My Drive/bird_classification//data/bird_dataset/test_images/mistery_category/0910f1d5badf42de1aa9bae05037412d.jpg',
 '/content/drive/My Drive/bird_classification//data/bird_dataset/test_images/mistery_category/09eb123fe843cf1ae1c14be61f55be37.jpg',
 '/content/drive/My Drive/bird_classification//data/bird_dataset/test_images/mistery_category/14938f3583bb58e008073772deb9832d.jpg',
 '/content/drive/My Drive/bird_classification//data/bird_dataset/test_images/mistery_category/14dfa660b37a9ee2712c38f9002d8980.jpg',
 '/content/drive/My Drive/bird_classification//data/bird_dataset/test_images/mistery_category/16f3532d0c649270ba1b6ee6a2a991d1.jpg',
 '/content/drive/My Drive/bird_classification//data/bird_dataset/test

In [ ]:
un_cropped

['/content/drive/My Drive/bird_classification//data/bird_dataset/test_images/mistery_category/0247efd7b9d47d036bb4390202a13e69.jpg',
 '/content/drive/My Drive/bird_classification//data/bird_dataset/test_images/mistery_category/0676b04fc9527343bee8ae5a2a9d4692.jpg',
 '/content/drive/My Drive/bird_classification//data/bird_dataset/test_images/mistery_category/4836bedeec2a2617ee33922ebf89995a.jpg',
 '/content/drive/My Drive/bird_classification//data/bird_dataset/test_images/mistery_category/64f3fa85502e9cff91d6dc88f54be7cb.jpg',
 '/content/drive/My Drive/bird_classification//data/bird_dataset/test_images/mistery_category/70fddb3c64b44a4d43e32a7c8148c425.jpg',
 '/content/drive/My Drive/bird_classification//data/bird_dataset/test_images/mistery_category/8ede0bc5a4976385dcfe6e38feaf90c2.jpg',
 '/content/drive/My Drive/bird_classification//data/bird_dataset/test_images/mistery_category/a05ed5dd6cbd3097e81e3c76ac690465.jpg',
 '/content/drive/My Drive/bird_classification//data/bird_dataset/test

In [ ]:
for namefile in os.listdir(orig_path + new_path):
  IMG = cv2.imread(orig_path + new_path + namefile)
  IMG = cv2.cvtColor(IMG, cv2.COLOR_BGR2RGB)
  cv2.imwrite(orig_path + new_path + namefile, IMG)

## Train Files

In [ ]:
stat = dict()

In [ ]:
path = 'train_images/'
new_path = 'train_images_cropped/'
for label in os.listdir(orig_path + 'train_images/'):
  os.makedirs(orig_path + new_path + label)

In [ ]:
for file_ in os.listdir(orig_path + 'train_images/'):
  path = 'train_images/' + file_ + '/'
  new_path = 'train_images_cropped/' + file_ + '/'
  i = 0
  for namefile in os.listdir(orig_path + path):
    current_img = orig_path + path + namefile
    try:
      cropped = crop_image(current_img, 0.55)
    except IndexError:
      print(f'No Bird detected on image {namefile}, trying lower threshold')
      try:
        cropped = crop_image(current_img, 0.40)
      except IndexError:
        continue
    if len(cropped) < 1:
      try:
        cropped = crop_image(current_img, 0.40)
      except IndexError:
        un_cropped.append(current_img)
        continue
      if len(cropped) < 1:
        un_cropped.append(current_img)
        continue 
    if len(cropped) > 1:
      print(f'Two bird detected on image {namefile}, trying bigger threshold')
      try:
        cropped = crop_image(current_img, 0.70)
      except IndexError:
        try:
          cropped = crop_image(current_img, 0.63)
        except IndexError:
          duplicates.append(current_img)
          continue
      if len(cropped) > 1:
        duplicates.append(current_img)
        continue
      elif len(cropped) < 1:
        cropped = crop_image(current_img, 0.63)
        if len(cropped) < 1 or len(cropped) > 2:
          duplicates.append(current_img)
          continue
    i += 1
    IMG = cropped[0]
    IMG = cv2.resize(IMG, (224,224), interpolation=cv2.INTER_LANCZOS4)
    IMG = cv2.cvtColor(IMG, cv2.COLOR_BGR2RGB)
    cv2.imwrite(orig_path + new_path + namefile, IMG)
  stat[file_] = (i, len(os.listdir(orig_path + path)))

In [ ]:
stat

{'004.Groove_billed_Ani': (48, 58),
 '009.Brewer_Blackbird': (52, 55),
 '010.Red_winged_Blackbird': (46, 53),
 '011.Rusty_Blackbird': (50, 57),
 '012.Yellow_headed_Blackbird': (40, 54),
 '013.Bobolink': (51, 58),
 '014.Indigo_Bunting': (47, 57),
 '015.Lazuli_Bunting': (50, 54),
 '016.Painted_Bunting': (48, 52),
 '019.Gray_Catbird': (50, 54),
 '020.Yellow_breasted_Chat': (43, 51),
 '021.Eastern_Towhee': (47, 54),
 '023.Brandt_Cormorant': (42, 53),
 '026.Bronzed_Cowbird': (47, 52),
 '028.Brown_Creeper': (48, 52),
 '029.American_Crow': (50, 53),
 '030.Fish_Crow': (49, 54),
 '031.Black_billed_Cuckoo': (39, 55),
 '033.Yellow_billed_Cuckoo': (48, 55),
 '034.Gray_crowned_Rosy_Finch': (46, 51)}

# Valid Files

In [ ]:
stat_valid = dict()

In [ ]:
path = 'val_images/'
new_path = 'val_images_cropped/'
for label in os.listdir(orig_path + 'val_images/'):
  os.makedirs(orig_path + new_path + label)

In [ ]:
for file_ in os.listdir(orig_path + 'val_images/'):
  path = 'val_images/' + file_ + '/'
  new_path = 'val_images_cropped/' + file_ + '/'
  i = 0
  for namefile in os.listdir(orig_path + path):
    current_img = orig_path + path + namefile
    try:
      cropped = crop_image(current_img, 0.55)
    except IndexError:
      print(f'No Bird detected on image {namefile}, trying lower threshold')
      try:
        cropped = crop_image(current_img, 0.40)
      except IndexError:
        continue
    if len(cropped) < 1:
      try:
        cropped = crop_image(current_img, 0.40)
      except IndexError:
        un_cropped.append(current_img)
        continue
      if len(cropped) < 1:
        un_cropped.append(current_img)
        continue 
    if len(cropped) > 1:
      print(f'Two bird detected on image {namefile}, trying bigger threshold')
      try:
        cropped = crop_image(current_img, 0.70)
      except IndexError:
        try:
          cropped = crop_image(current_img, 0.63)
        except IndexError:
          duplicates.append(current_img)
          continue
      if len(cropped) > 1:
        duplicates.append(current_img)
        continue
      elif len(cropped) < 1:
        cropped = crop_image(current_img, 0.63)
        if len(cropped) < 1 or len(cropped) > 2:
          duplicates.append(current_img)
          continue
    i += 1
    IMG = cropped[0]
    IMG = cv2.resize(IMG, (224,224), interpolation=cv2.INTER_LANCZOS4)
    IMG = cv2.cvtColor(IMG, cv2.COLOR_BGR2RGB)
    cv2.imwrite(orig_path + new_path + namefile, IMG)
  stat_valid[file_] = (i, len(os.listdir(orig_path + path)))

In [ ]:
stat_valid

{'004.Groove_billed_Ani': (1, 2),
 '009.Brewer_Blackbird': (3, 4),
 '010.Red_winged_Blackbird': (7, 7),
 '011.Rusty_Blackbird': (3, 3),
 '012.Yellow_headed_Blackbird': (2, 2),
 '013.Bobolink': (2, 2),
 '014.Indigo_Bunting': (3, 3),
 '015.Lazuli_Bunting': (4, 4),
 '016.Painted_Bunting': (6, 6),
 '019.Gray_Catbird': (5, 5),
 '020.Yellow_breasted_Chat': (6, 8),
 '021.Eastern_Towhee': (5, 6),
 '023.Brandt_Cormorant': (6, 6),
 '026.Bronzed_Cowbird': (8, 8),
 '028.Brown_Creeper': (7, 7),
 '029.American_Crow': (7, 7),
 '030.Fish_Crow': (4, 6),
 '031.Black_billed_Cuckoo': (3, 5),
 '033.Yellow_billed_Cuckoo': (3, 4),
 '034.Gray_crowned_Rosy_Finch': (7, 8)}